In [4]:
# general imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import pickle

# tensorflow imports
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, MaxPooling2D,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# tensorflow-privacy
import tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.membership_inference_attack as mia
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackInputData
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import SlicingSpec
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackType

import seaborn as sns
import json
import os
import pandas as pd
from pandas import DataFrame
from numpy.random import choice, seed
from numpy import ndarray, concatenate, stack, array, round, zeros, arange
import numpy as np

import category_encoders as ce
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder

import pickle

from os import mkdir, path

import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit

import warnings
warnings.filterwarnings('ignore')
from warnings import simplefilter
simplefilter('ignore', category=FutureWarning)
simplefilter('ignore', category=DeprecationWarning)

import scipy
print("scipy.__version__:  ", scipy.__version__)

import tensorflow_privacy.privacy
print("tensorflow_privacy.__version__:  ", tensorflow_privacy.__version__)

sns.set(style="whitegrid")


scipy.__version__:   1.11.1
tensorflow_privacy.__version__:   0.8.10


## 1. Read the input raw datasets 
#### 100 set of 1000 records each (= IN dataset for MIA RF-> get pred from classifier)
#### RAW data which which seeded the DP data. This will be needed for the MIA attack. 
#### But traning of the classifier will be done using the DP data set.

In [ ]:
filehandler = open("/home/privacy-master/DataShare/Adult_Raw_100set_1k.pkl","rb")
HundredSet=pickle.load(filehandler)
filehandler.close()
for i in range(len(HundredSet)):
    HundredSet[i]["income"] = np.where(HundredSet[i]['income'] == '<=50K' , 0, 1)

## 2. Read the TEST (OUT) set (= test set for the classifier; = test set for MIA RF)

In [4]:
filehandler = open(f'/home/privacy-master/DataShare/Adult_test5k_capped.pkl',"rb")
dfTest=pickle.load(filehandler)
filehandler.close()

dfTest ["income"] = np.where(dfTest['income'] == '<=50K' , 0, 1)
dfTest["income"].value_counts()

income
0    3761
1    1239
Name: count, dtype: int64

## 3. Read the synthetic sets (= train dataset for the classifier)

In [ ]:
epochs= ["500"]# ts for tabDDPM

san_k = epochs
dataFiles={}
for ts in epochs:
    filehandler = open(f'/home/privacy-master/DataShare/Adult_ddpm_500.pkl',"rb")
    dataFiles[ts]=pickle.load(filehandler)
    filehandler.close()

### Classifier begins

In [6]:
y_test = dfTest['income']
X_test = dfTest.drop(['income'], axis=1)

categorical = [col for col in X_test.columns if X_test[col].dtypes == 'O']
numerical = [col for col in X_test.columns if X_test[col].dtypes != 'O']

encoder = ce.OneHotEncoder(cols=['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'gender', 'native_country'])

encoder.fit(dfTest.drop(["income"], axis=1))
X_test = encoder.transform(X_test)

In [ ]:
utility_score={}
MIA_score={}
for epoc in epochs:
    utility_score[epoc]={}
    MIA_score[epoc]={}
    for set_no in range(len(HundredSet)):
        utility_score[epoc][set_no]=[]
        MIA_score[epoc][set_no]=[]
        
        dataFile = dataFiles[epoc][set_no]
        for serialID in range(2):#10

            df=dataFile[serialID]
            
            if (len(df)==0):
                continue
            count = df['income'].value_counts()
            #print(count)
            
            INset=HundredSet[set_no]

            y_train = df['income']
            X_train = df.drop(['income'], axis=1)
            #print(X_train.shape, X_test.shape)
            X_train = encoder.transform(X_train)
            
            
            INset_y_train = INset['income']
            INset_X_train = INset.drop(['income'], axis=1)
            INset_X_train = encoder.transform(INset_X_train)
                        

            cols = X_train.columns
            scaler = RobustScaler()
            X_test = scaler.fit_transform(X_test)
            X_train = scaler.transform(X_train)
            INset_X_train = scaler.transform(INset_X_train)

            X_train = pd.DataFrame(X_train, columns=[cols])
            X_test = pd.DataFrame(X_test, columns=[cols])
            INset_X_train = pd.DataFrame(INset_X_train, columns=[cols])
            

            # instantiate the classifier with n_estimators = 100
            # fit the model to the training set
            #rfc_100 = RandomForestClassifier(n_estimators=100, random_state=0)
            rfc_100=xgb.XGBClassifier()
            
            le = LabelEncoder().fit(y_train)
            y_train = le.transform(y_train)
            
            rfc_100.fit(X_train, y_train)
            # Predict on the test set results
            y_pred_100_enc = rfc_100.predict(X_test)
            y_pred_100 = le.inverse_transform(y_pred_100_enc)
            
            # Check accuracy score 
            acc = accuracy_score(y_test, y_pred_100)
            #print('Model accuracy score with epsilon {} DP: {:0.4f}'.format(epoc,acc))
            utility_score[epoc][set_no].append(acc)
            

            ## victim classifier has been trained with sanitized data.
        
            INx_train_pred1 = rfc_100.predict_proba(INset_X_train)
            OUTx_test_pred1 = rfc_100.predict_proba(X_test)
            
            
            
            INx_train_pred = np.zeros((len(INset_X_train), 2))
            INx_train_pred[0:len(INset_X_train), 0:INx_train_pred1.shape[1]]=INx_train_pred1
            OUTx_test_pred = np.zeros((len(X_test), 2))
            OUTx_test_pred[0:len(X_test), 0:OUTx_test_pred1.shape[1]] = OUTx_test_pred1
            
            
            probs_train = INx_train_pred
            probs_test = OUTx_test_pred
            #labels_train = y_train
            labels_train = np.zeros((len(INx_train_pred)))
            labels_train[0:len(INx_train_pred)] = INset_y_train
            labels_train = labels_train.astype(int)
            #labels_test = y_test
            labels_test = np.zeros((len(OUTx_test_pred)))
            labels_test[0:len(OUTx_test_pred)] = y_test
            labels_test=labels_test.astype(int)

            # define what variables our attacker should have access to
            attack_input = AttackInputData(
              probs_train = probs_train,
              probs_test = probs_test,
              #loss_train = loss_train,
              #loss_test = loss_test,
              labels_train = labels_train,
              labels_test = labels_test,
              multilabel_data = False#True
            )

            
            slicing_spec = SlicingSpec(
                entire_dataset = True,
                by_class = False,
                by_percentiles = False,
                by_classification_correctness = False)

            # define the type of attacker model that we want to use
            attack_types = [
                #AttackType.THRESHOLD_ATTACK,
                AttackType.RANDOM_FOREST#LOGISTIC_REGRESSION
            ]

            # run the attack
            attacks_result = mia.run_attacks(attack_input=attack_input,
                                             slicing_spec=slicing_spec,
                                             balance_attacker_training = False,
                                             attack_types=attack_types)

            #print(attacks_result.summary(by_slices=True))                        
            MIA_score[epoc][set_no].append(attacks_result.summary(by_slices=True))

In [8]:
print("Classification Accuracy: ")
for ts in epochs:
    print(f"eps = {ts}")
    all_id_sum=0
    for set_no in range(len(HundredSet)):
        all_id_sum+= ((sum(utility_score[ts][set_no]))/len(utility_score[ts][set_no]))
        print("%.2f"% ((sum(utility_score[ts][set_no]))/len(utility_score[ts][set_no])))
        print(len(utility_score[ts][set_no]))
    print("Average: %.2f"% (all_id_sum/len(HundredSet)))

Classification Accuracy: 
eps = 500
0.77
2
0.78
2
0.76
2
0.75
2
0.76
2
0.75
2
0.76
2
0.76
2
0.77
2
0.77
2
0.75
2
0.76
2
0.75
2
0.75
2
0.75
2
0.77
2
0.77
2
0.79
2
0.77
2
0.78
2
0.79
2
0.78
2
0.75
2
0.78
2
0.74
2
0.77
2
0.77
2
0.77
2
0.75
2
0.78
2
0.75
2
0.75
2
0.75
2
0.77
2
0.76
2
0.78
2
0.75
2
0.75
2
0.77
2
0.75
2
0.75
2
0.75
2
0.76
2
0.78
2
0.77
2
0.75
2
0.79
2
0.76
2
0.75
2
0.79
2
0.77
2
0.76
2
0.78
2
0.75
2
0.75
2
0.75
2
0.73
2
0.77
2
0.76
2
0.76
2
0.78
2
0.75
2
0.75
2
0.77
2
0.77
2
0.76
2
0.76
2
0.77
2
0.77
2
0.76
2
0.75
2
0.75
2
0.75
2
0.76
2
0.77
2
0.75
2
0.75
2
0.75
2
0.75
2
0.75
2
0.77
2
0.75
2
0.75
2
0.77
2
0.75
2
0.78
2
0.76
2
0.76
2
0.65
2
0.77
2
0.75
2
0.75
2
0.75
2
0.76
2
0.75
2
0.77
2
0.75
2
0.79
2
0.77
2
0.75
2
Average: 0.76


In [9]:
import re
print("MIA result: ")
for ts in epochs:
    print(f"eps = {ts}")
    all_id_AUCsum, all_id_ADVsum, all_id_PPVsum = 0, 0, 0
    
    for set_no in range(len(HundredSet)):#
        temp_AUCsum, temp_ADVsum, temp_PPVsum = 0, 0, 0
        for str1 in MIA_score[ts][set_no]:       
            result = re.findall(r"[-+]?\d*\.\d+|\d+", str1)
            temp_AUCsum += float(result[2])
            temp_ADVsum += float(result[5])
            temp_PPVsum += float(result[8])
        temp_AUCsum /= len(MIA_score[ts][set_no])#100
        temp_ADVsum /= len(MIA_score[ts][set_no])#100
        temp_PPVsum /= len(MIA_score[ts][set_no])#100
        print("AUC: %.2f \tADV: %.2f \tPPV: %.2f"% (temp_AUCsum, temp_ADVsum, temp_PPVsum))
        all_id_AUCsum += temp_AUCsum
        all_id_ADVsum += temp_ADVsum
        all_id_PPVsum += temp_PPVsum
    print("Average AUC: %.2f \tADV: %.2f \tPPV: %.2f"% (all_id_AUCsum/len(HundredSet), all_id_ADVsum/len(HundredSet), all_id_PPVsum/len(HundredSet)))

MIA result: 
eps = 500
AUC: 0.72 	ADV: 0.47 	PPV: 0.62
AUC: 0.96 	ADV: 0.84 	PPV: 0.98
AUC: 0.95 	ADV: 0.83 	PPV: 0.94
AUC: 0.96 	ADV: 0.86 	PPV: 0.98
AUC: 0.96 	ADV: 0.86 	PPV: 0.97
AUC: 0.97 	ADV: 0.89 	PPV: 0.98
AUC: 0.93 	ADV: 0.78 	PPV: 0.96
AUC: 0.97 	ADV: 0.92 	PPV: 0.98
AUC: 0.94 	ADV: 0.82 	PPV: 0.95
AUC: 0.96 	ADV: 0.86 	PPV: 0.98
AUC: 0.93 	ADV: 0.80 	PPV: 0.94
AUC: 0.95 	ADV: 0.81 	PPV: 0.97
AUC: 0.97 	ADV: 0.92 	PPV: 0.98
AUC: 0.96 	ADV: 0.85 	PPV: 0.97
AUC: 0.96 	ADV: 0.88 	PPV: 0.98
AUC: 0.95 	ADV: 0.83 	PPV: 0.95
AUC: 0.96 	ADV: 0.88 	PPV: 0.99
AUC: 0.95 	ADV: 0.83 	PPV: 0.96
AUC: 0.95 	ADV: 0.86 	PPV: 0.97
AUC: 0.92 	ADV: 0.76 	PPV: 0.94
AUC: 0.93 	ADV: 0.78 	PPV: 0.91
AUC: 0.92 	ADV: 0.73 	PPV: 0.95
AUC: 0.98 	ADV: 0.94 	PPV: 0.99
AUC: 0.96 	ADV: 0.84 	PPV: 0.97
AUC: 0.95 	ADV: 0.81 	PPV: 0.96
AUC: 0.95 	ADV: 0.81 	PPV: 0.98
AUC: 0.95 	ADV: 0.86 	PPV: 0.98
AUC: 0.95 	ADV: 0.83 	PPV: 0.99
AUC: 0.95 	ADV: 0.85 	PPV: 0.95
AUC: 0.96 	ADV: 0.88 	PPV: 0.98
AUC: 0.96 	ADV: 0